In [1]:
!pip install pennylane-lightning[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.0 MB/s eta 0:00:00


In [2]:
import pennylane as qml
import pennylane.numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import accuracy_score

# Load and normalize 25 features
data = load_breast_cancer()
X = data.data[:, :25]
y = data.target

# Split and normalize (L2 norm for amplitude encoding simulation)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = normalize(scaler.fit_transform(X_train), norm='l2')
X_test = normalize(scaler.transform(X_test), norm='l2')

# Data augmentation
X_train_aug = X_train + np.random.normal(0, 0.01, X_train.shape)
X_train = np.vstack([X_train, X_train_aug])
y_train = np.hstack([y_train, y_train])

# Simulated qPCA to reduce to 5D
# This follows the structure inspired by improved QPCA algorithms that use the density matrix
# as the covariance matrix to extract principal components efficiently.
def simulate_qpca(X, num_components=5):
    rho = sum(np.outer(x, x) for x in X) / len(X)  # Density matrix
    eigvals, eigvecs = np.linalg.eigh(rho)
    top_indices = np.argsort(eigvals)[-num_components:]
    top_vectors = eigvecs[:, top_indices]
    return X @ top_vectors

X_train = simulate_qpca(X_train, 5)
X_test = simulate_qpca(X_test, 5)

# Quantum device with 5 wires
dev = qml.device("lightning.gpu", wires=5)

# Variational circuit for 5 qubits
def variational_block(x, weights):
    for i in range(5):
        qml.RY(x[i], wires=i)
        qml.RZ(x[i] ** 2, wires=i)
    for i in range(5):
        qml.RY(weights[i], wires=i)
    for i in range(4):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[4, 0])

@qml.qnode(dev)
def quantum_node(x, weights):
    variational_block(x, weights)
    return [qml.expval(qml.PauliZ(i)) for i in range(5)]

# Hybrid model
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def leaky_relu(x, alpha=0.01):
    return np.where(x > 0, x, alpha * x)

def hybrid_model(x, params):
    q_out = np.array(quantum_node(x, params[:5]), requires_grad=True)
    hidden = leaky_relu(np.dot(params[5:10], q_out) + params[10])
    output = leaky_relu(np.dot(params[11:16], hidden) + params[16])
    final = sigmoid(np.dot(params[17:22], output) + params[22])
    return final

# Weighted BCE
def weighted_bce(preds, labels, pos_weight):
    return -np.mean(pos_weight * labels * np.log(preds + 1e-6) +
                    (1 - labels) * np.log(1 - preds + 1e-6))

# Cost
class_weights = np.array([1.0, len(y_train) / (2 * np.bincount(y_train)[1])])

def cost(params, X, y):
    preds = np.array([hybrid_model(x, params) for x in X], requires_grad=True)
    bce = weighted_bce(preds, y, pos_weight=class_weights[1])
    reg = 0.001 * np.sum(params ** 2)
    return bce + reg

def compute_accuracy(params, X, y):
    preds = [hybrid_model(x, params) > 0.5 for x in X]
    return accuracy_score(y, preds)

# Training
np.random.seed(42)
total_params = 23  # 5 (quantum) + 5+1 + 5+1 + 5+1
params = np.random.uniform(-0.1, 0.1, total_params, requires_grad=True)

opt = qml.AdamOptimizer(0.01)
epochs = 100
best_acc = 0
best_epo = 0

for epoch in range(epochs):
    params = opt.step(lambda p: cost(p, X_train, y_train), params)
    loss = cost(params, X_train, y_train)
    train_acc = compute_accuracy(params, X_train, y_train)
    test_acc = compute_accuracy(params, X_test, y_test)

    if test_acc > best_acc:
        best_acc = test_acc
        best_epo = epoch
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss:.4f} | Train Acc: {train_acc:.2f} | Test Acc: {test_acc:.2f}")

Epoch 0 | Loss: 0.6077 | Train Acc: 0.37 | Test Acc: 0.37
Epoch 10 | Loss: 0.6018 | Train Acc: 0.63 | Test Acc: 0.63
Epoch 20 | Loss: 0.5975 | Train Acc: 0.63 | Test Acc: 0.63
Epoch 30 | Loss: 0.5968 | Train Acc: 0.63 | Test Acc: 0.63
Epoch 40 | Loss: 0.5957 | Train Acc: 0.63 | Test Acc: 0.63
Epoch 50 | Loss: 0.5881 | Train Acc: 0.63 | Test Acc: 0.63
Epoch 60 | Loss: 0.5502 | Train Acc: 0.63 | Test Acc: 0.63
Epoch 70 | Loss: 0.4673 | Train Acc: 0.63 | Test Acc: 0.63
Epoch 80 | Loss: 0.3770 | Train Acc: 0.89 | Test Acc: 0.92
Epoch 90 | Loss: 0.3169 | Train Acc: 0.93 | Test Acc: 0.96


In [3]:
print(f"Best_acc :{best_acc:.2f} obtained on epoch: {best_epo}")
test_acc = compute_accuracy(params, X_test, y_test)
print(f"Current Test Acc after training for {epochs} : {test_acc:.2f}")

Best_acc :0.96 obtained on epoch: 91
Current Test Acc after training for 100 : 0.96
